In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_safety_stock_order_vendor (
  calday DATE,
  dc_id STRING,
  cnt_store BIGINT,
  product_id STRING,
  mch5_id STRING,
  asm_type STRING,
  start_date DATE,
  end_date DATE,
  normal_sale_qty DOUBLE,
  normal_sale_qty_acc DOUBLE,
  normal_sale_qty_dc DOUBLE,
  pct_acc DOUBLE,
  segment STRING,
  safety_stock BIGINT
)
USING DELTA
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);""")


In [0]:
current_7h = spark.sql("SELECT DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)").collect()[0][0]

if current_7h.day == 1:
    
    spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_safety_stock_order_vendor
        WHERE calday = DATE_ADD(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), -1)
    """)

    
    spark.sql(f"""
    INSERT INTO {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_safety_stock_order_vendor

    WITH table_date AS (
        SELECT 
            calday, 
            DATE_TRUNC('MONTH', calday) AS calmonth
        FROM (
            SELECT explode(sequence(
                add_months(date_trunc('MONTH', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), -8),
                date_sub(date_trunc('MONTH', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), 1)
            )) AS calday
        )
        WHERE 
            MONTH(calday) <> 11
            AND MONTH(calday) <> (
                SELECT MONTH(calday)
                FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_time_lunar_v2
                WHERE 
                    lunar_day = '1'
                    AND lunar_month = '1'
                    AND CAST(lunar_year AS INT) = YEAR(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS))
            )
    QUALIFY DENSE_RANK() OVER (ORDER BY month(calday) DESC) <= 6       
            ),
         
    store_sku_dc AS (
        SELECT 
            dc_id, store_id, product_id, 
            FIRST_VALUE(assortment_type) OVER (PARTITION BY dc_id, product_id ORDER BY business_unit DESC) AS asm_type,
            COUNT(store_id) OVER (PARTITION BY dc_id, product_id) AS cnt_store
        FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_biweekly
        WHERE dc_id IS NOT NULL
            AND business_unit IN ('1500', '2000')
            AND mdq > 0
            AND replenishment_mode = 'BS'
    ),
    base_sale AS (
        SELECT 
            b.dc_id,
            b.cnt_store,
            a.product_id,
            c.mch5_id,
            b.asm_type,
            MIN(a.calday) AS start_date,
            MAX(a.calday) AS end_date,
            SUM(
                GREATEST(
                    GREATEST(IFNULL(a.revenue.base_sale_qty, 0), 0)
                    - GREATEST(IFNULL(a.revenue.base_promotion_qty, 0), 0), 0
                )
            ) AS normal_sale_qty
        FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily a
        INNER JOIN store_sku_dc b ON a.store_id = b.store_id AND a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
        WHERE 
            a.calday >= DATE_SUB(DATE_TRUNC('MONTH', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), 8*30 )
            AND a.calday IN (SELECT calday FROM table_date)
        GROUP BY b.dc_id, b.cnt_store, a.product_id, c.mch5_id, b.asm_type
    ),
    seg AS (
        SELECT *,
            SUM(normal_sale_qty) OVER (PARTITION BY dc_id ORDER BY normal_sale_qty DESC) AS normal_sale_qty_acc,
            SUM(normal_sale_qty) OVER (PARTITION BY dc_id) AS normal_sale_qty_dc,
           (SUM(normal_sale_qty) OVER (PARTITION BY dc_id ORDER BY normal_sale_qty DESC) * 100
                / NULLIF(SUM(normal_sale_qty) OVER (PARTITION BY dc_id), 0)) AS pct_acc
        FROM base_sale
    )
    SELECT 
        DATE_ADD(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), -1) AS calday,
        *,
        CASE
            WHEN pct_acc < 40 AND asm_type IN ('Core 1', 'Core 2') THEN 'A'
            WHEN pct_acc >= 40 AND pct_acc < 80 AND asm_type IN ('Core 1', 'Core 2', 'Core 3') THEN 'B'
            ELSE 'C'
        END AS segment,
        CASE
            WHEN pct_acc < 40 AND asm_type IN ('Core 1', 'Core 2') THEN 4
            WHEN pct_acc >= 40 AND pct_acc < 80 AND asm_type IN ('Core 1', 'Core 2', 'Core 3') THEN 2
            ELSE 1
        END AS safety_stock
    FROM seg
    """)
